### Imports

In [2]:
from typing import List, Tuple, Dict

In [1]:
from config.settings import settings

### Document Loading

In [3]:
from src.ingestion.DocumentLoader import DocumentLoader

In [4]:
loader = DocumentLoader()

In [5]:
folder_name = "index"

In [6]:
files = loader.list_filenames(folder_name)
files

[METRICS] list_filenames: time=0.00s, count=1


['Graph_Databases_for_Beginners.pdf']

In [7]:
docs = loader.load_documents(subdir=folder_name,file_names=files)
# print(type(docs[0].page_content))

Files:   0%|          | 0/1 [00:00<?, ?file/s]

[METRICS] load_documents: time=2.80s, count=46


### Chunking

In [8]:
from src.ingestion.DocumentChunker import DocumentChunker

In [9]:
chunker = DocumentChunker(
    hf_embedding_model="sentence-transformers/all-mpnet-base-v2",
    chunk_size=300,
    chunk_overlap=80
)

In [10]:
chunks = chunker.chunk_documents(docs)
token_count = chunker.get_docs_token_count(chunks)

Chunking documents:   0%|          | 0/46 [00:00<?, ?doc/s]

[METRICS] chunk_documents: time=0.47s, count=88
[METRICS] get_docs_token_count: time=0.18s, count=88


In [11]:
print(len(chunks))
print(token_count)

88
26370


### Embedding

In [12]:
from src.ingestion.HuggingFaceEmbedder import HuggingFaceEmbedder

In [13]:
embedder = HuggingFaceEmbedder("sentence-transformers/all-mpnet-base-v2")

In [16]:
v1  = embedder.embed_query(chunks[0].page_content)
print("dimension",len(v1))

[METRICS] embed_query: time=0.12s, count=32
dimension 768


### Vector Store Management

In [17]:
from src.ingestion.VectorStoreManager import VectorStoreManager

In [18]:
vsm = VectorStoreManager(embedding_function=embedder,index_name=folder_name)

INFO:src.ingestion.VectorStoreManager:VectorStoreManager initialized for index 'index'


In [19]:
vsm.create_index()

INFO:src.ingestion.VectorStoreManager:Created FAISS index 'index' with dim=768


[METRICS] embed_query: time=0.13s, count=4


In [ ]:
vsm.add_documents(chunks)

In [ ]:
# vsm.save_local()

INFO:src.ingestion.VectorStoreManager:Saved index 'index' to '/home/ashmit/work/SEM_VIII/EnhancedRAG/context/faiss_indexes'


In [ ]:
# vsm.load_local(allow_pickle=True)

INFO:src.ingestion.VectorStoreManager:Loaded index 'index' from disk


### Retrieval

In [21]:
retrieved = vsm.similarity_search_with_score(query="experience at dolf", k=2)

[METRICS] embed_query: time=0.08s, count=6
[METRICS] similarity_search_with_score: time=0.09s, count=2


In [26]:
retriever = vsm.retriever(search_type = "similarity", search_kwargs = {"k":2})

INFO:src.ingestion.VectorStoreManager:Created retriever for 'index' with {'search_type': 'similarity', 'search_kwargs': {'k': 2}}


In [23]:
retrieved = retriever.invoke("projects by snakalp")

[METRICS] embed_query: time=0.17s, count=8


In [24]:
texts = [ret.page_content for ret in retrieved]
texts

['- time recommendations, graph - based search or supply - chain management, be sure to review all the different ways in which graph technology can work for your company. and while our customers span several continents and professional fields, they all agree that using the neo4j graph database is a critical component of their business success and competitiveness. are you a developer eager to learn more about making the switch? with so many ways to quickly get started, mastering graph database development is one of the best time investments you can make. other resources videos : • intro to neo4j and graph databases • intro to graph databases episode # 1 - evolution of dbs books : • o ’ reilly book : graph databases • learning neo4j trainings : • online training : getting started with neo4j • classroom trainings whether you need a solution that provides real - time recommendations, graph - based search or supply - chain management, be sure to review all the different ways in which graph 

### Supported LLMs (as of 06/05/2025)

In [ ]:
hf_llms=[
    "meta-llama/Llama-3.1-8B-Instruct",
    "meta-llama/Llama-3.3-70B-Instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    "mistralai/Mistral-7B-Instruct-v0.3",
    "nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
]

In [ ]:
from huggingface_hub import InferenceClient
from config.settings import settings

client = InferenceClient(
    provider="hf-inference",
    api_key=settings.HF_TOKEN.get_secret_value(),
)

def get_answer(
    sys_prompt: str,
    query: str,
    model: str = "nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
) -> str:
    """
    Send a system + user prompt to the specified model via HF Inference,
    returning the assistant’s content string.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user",   "content": query}
        ]
    )
    return response.choices[0].message.content


In [29]:
response = get_answer(sys_prompt="you are a helpful assistant who answers the users query concisely", query="what are the top houses in game of thrones")

ChatCompletionOutputMessage(role='assistant', content='A concise question about Westeros! Here are the **Top Houses in Game of Thrones**, in no particular order, along with their **Sigil**, **Motto**, and **Seat of Power**:\n\n1. **House Stark**\n\t* Sigil: Direwolf\n\t* Motto: "Winter is Coming"\n\t* Seat: Winterfell\n\n2. **House Lannister**\n\t* Sigil: Lion\n\t* Motto: "Hear Me Roar!"\n\t* Seat: Casterly Rock\n\n3. **House Targaryen**\n\t* Sigil: Dragon\n\t* Motto: "Fire and Blood"\n\t* Seat: Dragonstone (formerly King\'s Landing)\n\n4. **House Tyrell**\n\t* Sigil: Rose\n\t* Motto: "Growing Strong"\n\t* Seat: Highgarden\n\n5. **House Greyjoy**\n\t* Sigil: Kraken\n\t* Motto: "We Do Not Sow"\n\t* Seat: Pyke\n\n6. **House Baratheon**\n\t* Sigil: Stag\n\t* Motto: "Ours is the Fury"\n\t* Seats: Storm\'s End (original), King\'s Landing (during Robert\'s reign), Dragonstone (Stannis\' faction)\n\n7. **House Arryn**\n\t* Sigil: Falcon\n\t* Motto: "As High as Honor"\n\t* Seat: The Eyrie\n\n8

In [57]:
get_answer(sys_prompt="you are a helpful assistant who answers the users query concisely", query="what are the top houses in game of thrones")

'A concise question about Westeros! Here are the **Top Houses in Game of Thrones**, in no particular order, highlighting their **Sigil**, **Motto**, and **Notable Members**:\n\n1. **House Stark**\n\t* **Sigil**: Direwolf\n\t* **Motto**: "Winter is Coming"\n\t* **Notable Members**: Eddard (Ned), Robb, Sansa, Arya, Bran, Jon Snow\n\n2. **House Lannister**\n\t* **Sigil**: Lion\n\t* **Motto**: "Hear Me Roar!"\n\t* **Notable Members**: Cersei, Jaime, Tyrion, Tywin, Kevan\n\n3. **House Targaryen**\n\t* **Sigil**: Dragon\n\t* **Motto**: "Fire and Blood"\n\t* **Notable Members**: Daenerys, Viserys, Rhaegar, Aerys II (Mad King), Jon Snow (Aegon Targaryen)\n\n4. **House Baratheon**\n\t* **Sigil**: Stag\n\t* **Motto**: "Ours is the Fury"\n\t* **Notable Members**: Robert, Stannis, Renly, Joffrey, Myrcella, Tommen\n\n5. **House Tyrell**\n\t* **Sigil**: Rose\n\t* **Motto**: "Growing Strong"\n\t* **Notable Members**: Mace, Loras, Margaery, Olenna (Queen of Thorns)\n\n6. **House Greyjoy**\n\t* **Sigil

### Generation Pipeline

In [23]:
from src.generation.HuggingFaceLLM import HuggingFaceLLM

In [24]:
pg_llm = HuggingFaceLLM(model_name="meta-llama/Llama-3.1-8B-Instruct")

INFO:src.generation.HuggingFaceLLM:Creating new HuggingFaceLLM for model: meta-llama/Llama-3.1-8B-Instruct
INFO:src.generation.HuggingFaceLLM:InferenceClient initialized for model: meta-llama/Llama-3.1-8B-Instruct


In [29]:
# pg_llm.get_answer(sys_prompt="you are a helpful assistant that answers concisely", user_prompt="what is quantum computing ?", max_tokens = 200)

In [25]:
from src.generation.PromptAugmentor import PromptAugmentor

In [26]:
augmentor = PromptAugmentor(client=pg_llm)

In [27]:
prompts = augmentor.generate(query="what is a graph db and how is it different from a regular VectorDB ?", synthetic_count=4)
prompts

Generating prompts:   0%|          | 0/4 [00:00<?, ?prompt/s]

INFO:src.generation.PromptAugmentor:Requesting synthetic prompt 1/4
INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.1-8B-Instruct messages=2
INFO:src.generation.HuggingFaceLLM:get_answer returning 124 characters
INFO:src.generation.PromptAugmentor:Generated prompt #1: 'What is the key difference between a Graph Database and a Vector Database in terms of data structure and query capabilities?'
INFO:src.generation.PromptAugmentor:Requesting synthetic prompt 2/4
INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.1-8B-Instruct messages=2
INFO:src.generation.HuggingFaceLLM:get_answer returning 96 characters
INFO:src.generation.PromptAugmentor:Generated prompt #2: 'What are the primary use cases for Graph Databases compared to traditional relational databases?'
INFO:src.generation.PromptAugmentor:Requesting synthetic prompt 3

['What is the key difference between a Graph Database and a Vector Database in terms of data structure and query capabilities?',
 'What are the primary use cases for Graph Databases compared to traditional relational databases?',
 "What are the key advantages of using a Graph Database for complex relationships compared to a Vector Database's similarity-based connections?",
 'What are the primary data modeling and querying paradigms that distinguish a Graph Database from a Vector Database in terms of data representation and retrieval?',
 'what is a graph db and how is it different from a regular VectorDB ?']

In [42]:
from tqdm.auto import tqdm

prompt_chunks = []
for p in tqdm(prompts, desc="Retrieving chunks", unit="prompts"):
    docs = retriever.invoke(p)
    prompt_chunks.append((p, docs))


Retrieving chunks:   0%|          | 0/5 [00:00<?, ?prompts/s]

[METRICS] embed_query: time=0.28s, count=24
[METRICS] embed_query: time=0.22s, count=17
[METRICS] embed_query: time=0.19s, count=27
[METRICS] embed_query: time=0.24s, count=30
[METRICS] embed_query: time=0.34s, count=18


In [46]:
from src.generation.Fusion import FusionSummarizer
from src.generation.Prompts import Prompts

In [47]:
fusion_summarizer = FusionSummarizer(fusion_llm=pg_llm,sys_prompt=Prompts.MERGE_FUSION_SYS_PROMPT)

In [48]:
summaries = fusion_summarizer.summarize(prompt_chunks=prompt_chunks)

INFO:src.generation.Fusion:Generating summary for prompt 1: 'What is the key difference between a Graph Databas...'
INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.1-8B-Instruct messages=2
INFO:src.generation.HuggingFaceLLM:get_answer returning 1111 characters
INFO:src.generation.Fusion:Generating summary for prompt 2: 'What are the primary use cases for Graph Databases...'
INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.1-8B-Instruct messages=2
INFO:src.generation.HuggingFaceLLM:get_answer returning 1111 characters
INFO:src.generation.Fusion:Generating summary for prompt 3: 'what is a graph db and how is it different from a ...'
INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.1-8B-Instruct messages=2
INFO:src.generation.HuggingFace

In [41]:
all_summaries = "\n\n".join(summaries)

In [42]:
final_llm = HuggingFaceLLM(model_name="meta-llama/Llama-3.3-70B-Instruct")

INFO:src.generation.HuggingFaceLLM:Creating new HuggingFaceLLM for model: meta-llama/Llama-3.3-70B-Instruct
INFO:src.generation.HuggingFaceLLM:InferenceClient initialized for model: meta-llama/Llama-3.3-70B-Instruct


In [43]:
final_answer = final_llm.get_answer(sys_prompt=Prompts.FINAL_ANS_SYS_PROMPT,user_prompt="User Question: \nwhat is a graph db and how is it different from a regular VectorDB ? \n\n Context: \n"+all_summaries,max_tokens = 400, temperature = 0.7)

INFO:src.generation.HuggingFaceLLM:get_answer called
INFO:src.generation.HuggingFaceLLM:API call successful: model=meta-llama/Llama-3.3-70B-Instruct messages=2
INFO:src.generation.HuggingFaceLLM:get_answer returning 599 characters


In [45]:
print(final_answer)

A graph database is an online, operational database management system that operates on a graph data model, storing data as nodes and relationships. It is different from a regular VectorDB in terms of data model, storage, and processing. Graph databases use a graph data model, native graph storage, and native graph processing, whereas VectorDBs are suited for dense, high-dimensional data and are often used in scenarios such as recommendation systems, image and speech recognition, and Natural Language Processing (NLP). Insufficient information is available to provide a more detailed comparison.
